# 05 — Market Value Time Series
**Requires:** Run `01_load_and_filter.ipynb` first.

How has the market value of Greek players evolved over time? Tracks individual trajectories and overall trends.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

OUTPUTS_PATH = "../outputs/"
FIGURES_PATH = "../outputs/figures/"

greek_valuations = pd.read_parquet(OUTPUTS_PATH + "greek_valuations.parquet")
greek_all        = pd.read_parquet(OUTPUTS_PATH + "greek_all.parquet")

# Join player name and position into valuations
greek_valuations = greek_valuations.merge(
    greek_all[["player_id", "name", "position", "highest_market_value_in_eur"]],
    on="player_id",
    how="left"
)
greek_valuations["year"] = greek_valuations["date"].dt.year

print("greek_valuations:", greek_valuations.shape)
greek_valuations.head()

## 5.1 Median Market Value of All Greek Players Over Time

In [ ]:
avg_by_year = (
    greek_valuations
    .groupby("year")["market_value_in_eur"]
    .agg(median="median", mean="mean", count="count")
    / 1e6
)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

avg_by_year["median"].plot(ax=axes[0], marker="o", color="steelblue")
axes[0].set_title("Median Market Value of Greek Players Over Time")
axes[0].set_ylabel("Median Value (€M)")
axes[0].set_xlabel("Year")

avg_by_year["count"].plot(ax=axes[1], marker="o", color="gray")
axes[1].set_title("Number of Valuation Records per Year")
axes[1].set_ylabel("Count")
axes[1].set_xlabel("Year")

plt.tight_layout()
plt.savefig(FIGURES_PATH + "05_median_value_over_time.png", dpi=150)
plt.show()

## 5.2 Value by Position Over Time

In [ ]:
value_by_pos_year = (
    greek_valuations
    .groupby(["year", "position"])["market_value_in_eur"]
    .median()
    .unstack()
    / 1e6
)

value_by_pos_year.plot(figsize=(12, 5), marker="o")
plt.title("Median Market Value by Position Over Time")
plt.ylabel("Median Value (€M)")
plt.xlabel("Year")
plt.legend(title="Position")
plt.tight_layout()
plt.savefig(FIGURES_PATH + "05_value_by_position_over_time.png", dpi=150)
plt.show()

## 5.3 Top 5 Greek Players — Individual Value Trajectories

In [ ]:
# Get top 5 by peak value
top5 = (
    greek_all[["player_id", "name", "highest_market_value_in_eur"]]
    .dropna(subset=["highest_market_value_in_eur"])
    .sort_values("highest_market_value_in_eur", ascending=False)
    .head(5)
)

print(top5[["name", "highest_market_value_in_eur"]].to_string(index=False))

plt.figure(figsize=(12, 5))
for _, row in top5.iterrows():
    data = (
        greek_valuations[greek_valuations["player_id"] == row["player_id"]]
        .sort_values("date")
    )
    if len(data) > 0:
        plt.plot(data["date"], data["market_value_in_eur"] / 1e6,
                 marker=".", label=row["name"])

plt.title("Market Value Trajectory — Top 5 Greek Players")
plt.ylabel("Market Value (€M)")
plt.xlabel("Date")
plt.legend()
plt.tight_layout()
plt.savefig(FIGURES_PATH + "05_top5_trajectories.png", dpi=150)
plt.show()

## 5.4 Peak Age — At What Age Do Greek Players Reach Their Highest Value?

In [ ]:
# Join date_of_birth to valuations
greek_valuations_age = greek_valuations.merge(
    greek_all[["player_id", "date_of_birth"]],
    on="player_id",
    how="left"
)
greek_valuations_age["date_of_birth"] = pd.to_datetime(greek_valuations_age["date_of_birth"])
greek_valuations_age["age_at_valuation"] = (
    (greek_valuations_age["date"] - greek_valuations_age["date_of_birth"]).dt.days // 365
)

# Find peak value record per player
peak_ages = (
    greek_valuations_age
    .loc[greek_valuations_age.groupby("player_id")["market_value_in_eur"].idxmax()]
    [["player_id", "name", "age_at_valuation", "market_value_in_eur"]]
    .dropna(subset=["age_at_valuation"])
)

plt.figure(figsize=(10, 4))
peak_ages["age_at_valuation"].plot(kind="hist", bins=20, color="gold", edgecolor="black")
plt.axvline(peak_ages["age_at_valuation"].median(), color="red", linestyle="--",
            label=f'Median peak age: {peak_ages["age_at_valuation"].median():.0f}')
plt.title("At What Age Do Greek Players Reach Peak Value?")
plt.xlabel("Age")
plt.legend()
plt.tight_layout()
plt.savefig(FIGURES_PATH + "05_peak_age.png", dpi=150)
plt.show()

print(peak_ages["age_at_valuation"].describe())